# **Sparked_Internship_Assignment**

### I just want to mention few things 
- I have increased loyalty_score by 1, everytime a purchase is made.
- I can change it to some formula if you want me to do so.
- I have put (***) sign before whenever i have used show to see a new dataframe and i have commented the show function so if you want to see the dataframe just uncomment it.I thought during some point of time you want to see what the dataframe contains instead of typing show you can just uncomment it.
- I have put all the three files in the same folder, if you want me to put it somewhere else and then generating the datefarm from it, I can do that. 
- I have not written the dataframe out to a csv file as I was not clear whether I should do it or not, if you want me to write it, I can do that





In [1]:
## importing modules


import findspark
from pyspark import SparkContext
from pyspark.sql import SparkSession, Window, Row
from pyspark.sql.functions import *
from pyspark.sql.types import *
import matplotlib.pyplot as plt
import ipytest
ipytest.autoconfig()

In [2]:
## entering into Apache Spark Ecosystem

spark = SparkSession.builder.appName("firstSpark").getOrCreate()


In [3]:
## function to create a dataframe
def load_dataframe(filename, format):
    if format == 'csv':
        df = spark.read.format('csv').options(header='true').load(filename)
        return df
    if format == 'json':
        df = spark.read.option('multiline',True).json('transactions.json')
        return df


### dateframe of all the three files 
df_customers = load_dataframe('customers.csv','csv')
df_products = load_dataframe('products.csv','csv')
df_transactions = load_dataframe('transactions.json','json')


In [4]:
## renaming some columns
df_customers= df_customers.withColumnRenamed('customer_id','customer_id_customer')
df_products = df_products.withColumnRenamed('product_id','product_id_product')

#### ***

In [5]:
# df_customers.show()
# df_products.show()


In [6]:
### joining transaction dataframe and customers dataframe and storing it as a new dataframe  

df_new1 = df_transactions.join(df_customers,df_transactions.customer_id == df_customers.customer_id_customer).select(['customer_id','loyalty_score','basket','date_of_purchase'])

#### ***

In [7]:
# df_new1.show()

In [8]:
### ordering the new dateframe by time of purchase

df_new1 = df_new1.orderBy(col('date_of_purchase').desc())

In [9]:
### calculating the size of basket in transaction.json file

df_new1 = df_new1.withColumn('basket_size',size(col('basket')))

In [10]:
### updating the loyalty increasing it by 1 when a purchase is made

df_new1 = df_new1.withColumn('loyalty_score',col('loyalty_score')+col('basket_size'))

#### ***

In [11]:
# df_new1.show()

In [12]:
### exploding the basket map using the explode fucntion provided in pyspark library

df_new1 = df_new1.select(df_new1.customer_id,df_new1.loyalty_score,explode(df_new1.basket),df_new1.date_of_purchase)

#### ***

In [13]:
# df_new1.show()

In [14]:
### extracting the product_id with the help of getItem

df_new1 = df_new1.withColumn('product_id',df_new1['col'].getItem('product_id'))

In [15]:
### Dropping the column 'col'
df_new1 = df_new1.drop('col')

#### ***

In [16]:
# df_new1.show()

In [17]:
### joining the products dataframe with df_new1 dataframe

df_new2 = df_new1.join(df_products,df_new1.product_id == df_products.product_id_product).select(['customer_id','loyalty_score','date_of_purchase','product_id','product_category'])

#### ***

In [18]:
## df_new2.show()

In [19]:
### counting the number of distinct products bought

df_new3 = df_new2.groupBy('product_id').count()

#### ***

In [20]:
## df_new3.show()

In [21]:
### renaming columns product_id and count in df_new3

df_new3 = df_new3.withColumnRenamed('product_id','product_id_count')
df_new3 = df_new3.withColumnRenamed('count','count_of_id')

#### ***

In [22]:
## df_new3.show()

In [23]:
### joining the dataframe which contains the purchase count with the one which contains updated loyalty

df_final = df_new2.join(df_new3,df_new2.product_id == df_new3.product_id_count).select(['customer_id','loyalty_score','date_of_purchase','product_id','product_category','count_of_id'])

#### ***

In [24]:
## df_final.show()

In [25]:
### ordering the dataframe by time of purchase

df_final = df_final.orderBy(col('date_of_purchase').desc())

#### ***

In [26]:
### df_final.show()

In [27]:
### renaming the column count_of_id to purchase_count_per_product_id

df_final = df_final.withColumnRenamed('count_of_id','purchase_count_per_product_id')

#### ***

In [28]:
### df_final.show()

**The reason I have used distinct fucntion here is, when there are two of same products in a customer's basket, it results into a duplicate dataframe row so to counter it, I have used distinct**

In [29]:
### removing duplicate rows which happens when a customer buys more than 1 of some product

df_final_distinct = df_final.distinct()

#### ***

In [30]:

### df_final_distinct.show()

In [31]:
### ordering the dataframe by date of purchase

df_final_distinct = df_final_distinct.orderBy(col('date_of_purchase').asc())

#### ***

In [32]:
### df_final_distinct.show()

In [33]:
### dropping the date_of_purchase column as it is not asked in the challenge

df_final_distinct = df_final_distinct.drop('date_of_purchase')

In [34]:
### displaying the final output

df_final_distinct.show()

+-----------+-------------+----------+----------------+-----------------------------+
|customer_id|loyalty_score|product_id|product_category|purchase_count_per_product_id|
+-----------+-------------+----------+----------------+-----------------------------+
|         C1|         13.0|        P1|               1|                            8|
|         C1|         13.0|        P3|               3|                            1|
|         C2|        234.0|        P1|               1|                            8|
|         C2|        234.0|        P2|               2|                            6|
|         C3|         25.0|        P1|               1|                            8|
|         C3|         25.0|        P2|               2|                            6|
|         C4|         16.0|        P1|               1|                            8|
|         C4|         16.0|        P2|               2|                            6|
|         C5|         54.0|        P1|               1

**TEST CASES**

In [40]:
%%ipytest

## checking the customers dataframe
def test_customers_1():
    row =  df_customers.count() 
    assert row == 7
def test_customers_2():
    col = len(df_customers.dtypes)
    assert col == 2
 ## checking for null values in customers dataframe   
def test_customers_null():
    assert df_customers.filter(df_customers.customer_id_customer.isNull() | df_customers.loyalty_score.isNull()).count() == 0    
    
    
## checking the products dataframe 
def test_products_1():
    row =  df_products.count() 
    assert row == 3
def test_products_2():
    col = len(df_products.dtypes)
    assert col == 3
 ## checking for null values in products_dataframe   
def test_products_null():
    assert df_products.filter(df_products.product_id_product.isNull() | df_products.product_description.isNull() | df_products.product_category.isNull()).count() == 0 
 
 ## checking the final dataframe 
def test_final_distinct_1():    
    assert len(df_final_distinct.dtypes) == 5
 ## checking for null values in final_dataframe   
def test_final_distinct_null():
    assert df_final_distinct.filter(df_final_distinct.customer_id.isNull() | df_final_distinct.loyalty_score.isNull() | df_final_distinct.product_category.isNull() | df_final_distinct.product_id.isNull() | df_final_distinct.purchase_count_per_product_id.isNull()).count() == 0 
 


........                                                                                     [100%]
8 passed in 0.98s
